In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 3.8MB 20.6MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 512kB 39.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=784bcef797eae97aa4b913e7a18bec2afe7f5e59436908d4b3abef4b3795a80c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [3]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [6]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-use-rdr-4500-1-1-1-ver2.h5")

In [7]:
import numpy as np
from numpy import argmax

In [8]:
char_encode = np.loadtxt('/content/drive/MyDrive/fast_text/val/char_encode.txt')
print(char_encode.shape)
char_encode = char_encode.reshape(int(char_encode.shape[0]/42), 42, char_encode.shape[1])
print(char_encode.shape)

(18900, 32)
(450, 42, 32)


In [9]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

(450, 42, 32, 137)


In [10]:
tag = np.loadtxt('/content/drive/MyDrive/fast_text/val/tag_embedd.txt')
tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
print(tag.shape)

(450, 42, 13)


In [11]:
word = np.loadtxt('/content/drive/MyDrive/fast_text/val/word_embedd.txt')
word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
print(word.shape)

(450, 42, 300)


In [12]:
y_pred = m.predict([char_embedd, word])
y_true = tag
print(y_pred.shape)
print(y_true.shape)

(450, 42, 13)
(450, 42, 13)


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
print(y_pred.shape)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred.shape)
y_pred = y_pred.reshape(int(y_pred.shape[0]/42), 42)
print(y_pred.shape)

(18900, 13)
(18900,)
(450, 42)


In [15]:
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
print(y_true.shape)
y_true = np.argmax(y_true,axis=1)
print(y_true.shape)
y_true = y_true.reshape(int(y_true.shape[0]/42), 42)
print(y_true.shape)

(18900, 13)
(18900,)
(450, 42)


In [16]:
from sklearn.metrics import classification_report, confusion_matrix

In [17]:
def arr_tag_in_a_tag_sen(sen_pred, sen_true, tag):
  j=0
  bat_dau=-1
  ket_thuc=-1
  while j < len(sen_pred):
    if sen_true[j]==tag:
      bat_dau=j
      ket_thuc=j
      while sen_true[ket_thuc+1]==1:
        ket_thuc=ket_thuc+1
      ket_thuc=ket_thuc+1
      break
    else:
        j=j+1

  if bat_dau==-1 and ket_thuc==-1:
    return 0
  
  a = sen_pred[bat_dau:ket_thuc]
  b = sen_true[bat_dau:ket_thuc]
  a = np.array(a)
  b = np.array(b)
  if (a==b).all():
    return 1
  else:
    return 0

In [18]:
from sklearn.metrics import accuracy_score
acc=0
false=0

homenumber_true = 0
street_true = 0 
ward_true = 0
district_true = 0
province_true = 0
country_true = 0
postcode_true = 0
ner_true = 0
obj_true = 0
obj_feature_true = 0
pre_true = 0

homenumber_false = 0
street_false = 0
ward_false = 0
district_false = 0
province_false = 0
country_false = 0
postcode_false = 0
ner_false = 0
obj_false = 0
obj_feature_false = 0
pre_false = 0

In [19]:
for i in range(len(y_true)):
  sen_pred = y_pred[i]
  sen_true = y_true[i]
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 2)==1:
    street_true+=1
  else:
    street_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 3)==1:
    ward_true+=1
  else:
    ward_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 4)==1:
    district_true+=1
  else:
    district_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 5)==1:
    province_true+=1
  else:
    province_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 6)==1:
    country_true+=1
  else:
    country_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 7)==1:
    postcode_true+=1
  else:
    postcode_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 8)==1:
    ner_true+=1
  else:
    ner_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 9)==1:
    obj_true+=1
  else:
    obj_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 10)==1:
    obj_feature_true+=1
  else:
    obj_feature_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 11)==1:
    pre_true+=1
  else:
    pre_false+=1

In [20]:
acc = homenumber_true+ street_true+ ward_true+ district_true+ province_true+ country_true+ postcode_true+ ner_true+ obj_true+ obj_feature_true+ pre_true
false = homenumber_false+ street_false+ ward_false+ district_false+ province_false+ country_false+ postcode_false+ ner_false+ obj_false+ obj_feature_false+ pre_false
print(acc)
print(false)
print('accuracy = {}'.format(acc/(acc+false)) )
ti_le_dung = format(acc/(acc+false)*100, '.2f')
print(ti_le_dung)

print("homenumber : {} true, {} false".format(homenumber_true, homenumber_false))
print("street : {} true, {} false".format(street_true, street_false))
print("ward : {} true, {} false".format(ward_true, ward_false))
print("district : {} true, {} false".format(district_true, district_false))
print("province : {} true, {} false".format(province_true, province_false))
print("country : {} true, {} false".format(country_true, country_false))
print("postcode : {} true, {} false".format(postcode_true, postcode_false))
print("ner : {} true, {} false".format(ner_true, ner_false))
print("obj : {} true, {} false".format(obj_true, obj_false))
print("obj_feature : {} true, {} false".format(obj_feature_true, obj_feature_false))
print("pre : {} true, {} false".format(pre_true, pre_false))


2832
2568
accuracy = 0.5244444444444445
52.44
homenumber : 462 true, 438 false
street : 293 true, 157 false
ward : 155 true, 295 false
district : 300 true, 150 false
province : 300 true, 150 false
country : 142 true, 308 false
postcode : 300 true, 150 false
ner : 138 true, 312 false
obj : 296 true, 154 false
obj_feature : 161 true, 289 false
pre : 285 true, 165 false


In [21]:
dict = []
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1])
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1])
for element in y_true:
  if element not in dict:
    dict.append(element)
# print(dict)
dict.sort()
print(dict)

cnf_matrix = confusion_matrix(y_true, y_pred, labels=dict)
for i in range(len(cnf_matrix)):
  for j in range(len(cnf_matrix)):
    cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
print(cnf_matrix)

arr_to_cal_predict = []
for i in range(len(cnf_matrix)):
  tong= 0
  for j in range(len(cnf_matrix)):
    tong+=cnf_matrix[j][i]
  arr_to_cal_predict.append(tong)
print(arr_to_cal_predict)
# lấy độ predict
predict = []
for i in range(len(cnf_matrix)):
  tong=cnf_matrix[i][i]/arr_to_cal_predict[i]
  predict.append(tong)

print("Độ đo P của từng tag: ")
print(predict)
print(len(predict))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[[13501     0     0     0     0     0     0     0     0     0     0     0
      0]
 [    0   301     4     0     0     0     0     0     0     0     0     2
      0]
 [    0     5  1105     2     0     0     0     0     0     0     0     1
      0]
 [    0     0     0   155     0     0     0     0     0     0     0     0
      0]
 [    0     0     0     0   300     0     0     0     0     0     0     0
      0]
 [    0     0     0     0     0   300     0     0     0     0     0     0
      0]
 [    0     0     0     0     0     0   142     0     0     0     0     0
      0]
 [    0     0     0     0     0     0     0   300     0     0     0     0
      0]
 [    3     0     0     0     0     0     1     2   484     4     3    12
      6]
 [    0     0     0     0     0     0     0     0     8   555    11     0
      1]
 [    0     2     1     0     0     0     0     0     4    31   330     9
      3]
 [    0     0     2     0     0     0     0 

In [22]:
normalized_confusion_matrix = cnf_matrix/cnf_matrix.sum(axis = 1, keepdims = True)
for i in range(len(normalized_confusion_matrix)):
  for j in range(len(normalized_confusion_matrix)):
    normalized_confusion_matrix[i][j] = round(normalized_confusion_matrix[i][j], 3)
print(normalized_confusion_matrix)

# lấy độ recall
recall = []
for i in range(len(normalized_confusion_matrix)):
  recall.append(normalized_confusion_matrix[i][i])

print("Độ đo R của từng tag: ")
print(recall)
print(len(recall))

[[1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.98  0.013 0.    0.    0.    0.    0.    0.    0.    0.    0.007
  0.   ]
 [0.    0.004 0.993 0.002 0.    0.    0.    0.    0.    0.    0.    0.001
  0.   ]
 [0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.
  0.   ]
 [0.006 0.    0.    0.    0.    0.    0.002 0.004 0.94  0.008 0.006 0.023
  0.012]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.014 0.965 0.019 0.
  0.002]
 [0.    0.005 0.003 0.    0.    0.    0.    0.    0.011 0.082 0.868 0.024
  0.008]
 [0.    0.    0.004 0.    0.    0.    0.    0.    0.01  0.    0.006 0.969
  0.01 ]
 [0.    0.    0.001 0.   

In [23]:
# lấy độ F1
f1 = []
for i in range(len(recall)):
  p = predict[i]
  r = recall[i]
  f1.append(2*p*r/(p+r))

print("Độ đo F1 của từng tag: ")
print(f1)
print(len(f1))

Độ đo F1 của từng tag: 
[0.9998889094612108, 0.9786344635392475, 0.9929061007352699, 0.9935897435897436, 1.0, 1.0, 0.9964912280701755, 0.9966777408637874, 0.9528556767964479, 0.9526837728465336, 0.9076103143872901, 0.9470121372031662, 0.9825177275788409]
13
